In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
def convert_list_to_in_sql(src):
    return str(src).replace("[", "(").replace("]", ")")

bigquery-public-data.stackoverflow.posts_questions

In [2]:
# Connect to stackoverflow dataset
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
stackOverflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                         dataset_name="stackoverflow")

Using Kaggle's public dataset BigQuery integration.


In [3]:
# Get bq assistant for data
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")

Using Kaggle's public dataset BigQuery integration.


In [4]:
# Select random users, user(id, display_name, creation_date, last_access_date, reputation)
myusers = bq_assistant.head("users", num_rows = 10000).sample(n = 10)[["id", "display_name", "creation_date", "last_access_date"]]

# Renaming columns
myusers.rename(columns = {"display_name": "name", "creation_date" : "registration_timestrmpz", "last_access_date" : "last_access_timestrmpz"}, inplace = True)

# Exporting to csv
myusers.to_csv("users.csv", index = False)

In [5]:
# Id list of my users
id_of_myusers = myusers["id"].values.tolist()
in_sql_id_of_myusers = convert_list_to_in_sql(id_of_myusers)

In [6]:
# Select all posts of my users (post_questions in original data)
posts_questions_of_myusers = bq_assistant.query_to_pandas("SELECT * FROM bigquery-public-data.stackoverflow.posts_questions WHERE owner_user_id IN " + in_sql_id_of_myusers)

# Select id, title, body, owner_user_id, creation_date, last_activity_date
posts_questions_of_myusers = posts_questions_of_myusers[["id", "title", "body", "owner_user_id", "creation_date", "last_activity_date"]]

# Rename columns
posts_questions_of_myusers.rename(columns = {"owner_user_id" : "user_creator_id", "creation_date" : "creation_timestrmpz", "last_activity_date" : "last_activity_timestrmpz"}, inplace = True)

# Export to csv
posts_questions_of_myusers.to_csv("posts.csv", index = False)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


In [7]:
# Id list of my posts
id_of_posts_questions_of_myusers = posts_questions_of_myusers["id"].values.tolist()
in_sql_id_of_posts_questions_of_myusers = convert_list_to_in_sql(id_of_posts_questions_of_myusers)

In [8]:
# Select all comments of my users for users and posts
comments_of_myusers = bq_assistant.query_to_pandas("SELECT * FROM bigquery-public-data.stackoverflow.comments WHERE user_id IN " + in_sql_id_of_myusers + " AND post_id IN " + in_sql_id_of_posts_questions_of_myusers)

# Select id, text, post_id, user_id, creation_date
comments_of_myusers = comments_of_myusers[["id", "text", "post_id", "user_id", "creation_date"]]

# Renaming columns
comments_of_myusers.rename(columns = {"text" : "body", "user_id" : "user_creator_id", "creation_date" : "creation_timestrmpz"}, inplace = True)

# Exporting to csv
comments_of_myusers.to_csv("comments.csv", index = False)

In [9]:
# Interesting fact: votes DO NOT HAVE link to user. So votes are anonymous!!!

# So I decided to get 1451 random votes from first 10000 votes
votes = bq_assistant.head("votes", num_rows = 10000).sample(n = 1451)[["id", "vote_type_id", "creation_date"]]

# Renaming columns
votes.rename(columns = {"creation_date" : "creation_timestrmpz"}, inplace = True)

# Exporting to csv
votes.to_csv("votes.csv", index = False)